In [20]:
import uNet1D as unet
import importlib
importlib.reload(unet)
#unet=unet.unet1D(input_size = (48,3))
#print(unet.summary())
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras.layers import Input, concatenate
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import models
from tensorflow.keras import utils
from tensorflow import keras

def get_model_bce(nout,ninput=110):
    model = keras.Sequential([
        layers.Dense(30, activation='relu', input_shape=[ninput]),
        layers.Dropout(0.1),
        layers.Dense(30, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(30, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(nout, activation='softmax')
    ])
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss=losses.CategoricalCrossentropy(),
        metrics=['accuracy'])
    return model

lidar_model=get_model_bce(17,ninput=52)
radar_model=get_model_bce(17,ninput=50)
lidarRadar_model=get_model_bce(17,ninput=102)
radiometer_model=get_model_bce(17,ninput=10)
lidarRadarRadiometer_model=get_model_bce(17,ninput=112)



In [4]:
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
from sklearn.model_selection import train_test_split
from netCDF4 import Dataset

with Dataset("simObsCrossTrack_2dn_extended.nc") as fh:
    yobsL=fh["tb"][:,:]
    x2D=fh["iwc_SSRG"][:,:]
    iwc_CS=fh["iwcCS"][:,:]
    zKu=fh["zKu"][:,:]
    pnorm=fh["pnorm"][:,:]
    pmol=fh["pmol"][:,:]

#set random seed
np.random.seed(42)
nt,nchan=yobsL.shape   
lidarNoise=np.random.randn(nt,50)*0.1
pnorm=np.log10(1e-3*(pnorm)*10**lidarNoise+1e-9)
pnorm[pnorm<-8.0]=-8.0
pnormMax=pnorm.max(axis=-1)
nt,nchan=yobsL.shape
lidarEchoPeak=[]
zKuGtL=[]
#print(zKu.shape)
#stop
zKu+=np.random.randn(nt,50)
zKu[zKu<8]=0
# do pca on zKu
from sklearn.decomposition import PCA
bscatter=pnorm[:,30:45]
for i,pnorm1 in enumerate(pnorm):
    lidarEchoPeak.append([np.argmax(pnorm1)*0.24,2*pnorm1.max()])
    z1=zKu[i]
    a=np.nonzero(z1>8)
    if len(a[0])>0:
        zKuGtL.append(z1)
    else:
        zKuGtL.append(z1)
yobsL+=np.random.randn(nt,nchan)*3
zKuGtL=np.array(zKuGtL)
yobsRadiomLidar=np.concatenate((yobsL,pnorm),axis=-1)
yobsRadiomLidarRadar=np.concatenate((yobsL,lidarEchoPeak,pnorm,zKuGtL),axis=-1)
ind_train,ind_test,y_train, y_test = train_test_split(range(nt), x2D[:].sum(axis=-1)*0.24, test_size=0.33, random_state=42)
nt,nc=yobsL.shape

X1_train=yobsRadiomLidarRadar[ind_train,:]
X1_test=yobsRadiomLidarRadar[ind_test,:]
iwc_train=x2D[ind_train,:]
iwc_test=x2D[ind_test,:]
import pickle
with open('kmeans_iwc.pklz','rb') as f:
    kmeans_iwc=pickle.load(f)["kmeans_iwc"]

iwcCS_train=iwc_CS[ind_train,:]
iwcCS_test=iwc_CS[ind_test,:]
iwc_float=iwcCS_train[:,:].data.astype(np.float16)
labels_=kmeans_iwc.predict(iwc_float)
print(X1_train.shape)
print(iwc_train.shape)
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scalerIWC = StandardScaler()
a=list(range(1,49))
scalerIWC.fit(iwc_train[:,a])
iwc_train_scaled=scalerIWC.transform(iwc_train[:,a])
iwc_test_scaled=scalerIWC.transform(iwc_test[:,a])
print(iwc_train_scaled.shape)
nC=18
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X1_train)
X_train_scaled=scaler.transform(X1_train)
X_test_scaled=scaler.transform(X1_test)
print(X_train_scaled.shape)

(561647, 112)
(561647, 50)
(561647, 48)
(561647, 112)


In [22]:

import pickle
with open('kmeans_iwc.pklz','rb') as f:
    kmeans_iwc=pickle.load(f)["kmeans_iwc"]

iwcCS_train=iwc_CS[ind_train,:]
iwcCS_test=iwc_CS[ind_test,:]
iwc_float=iwcCS_train[:,:].data.astype(np.float16)
labels_train=kmeans_iwc.predict(iwc_float)
a=np.nonzero(labels_train!=0)
labels_test=kmeans_iwc.predict(iwcCS_test[:,:].data.astype(np.float16))
b=np.nonzero(labels_test!=0)
print(len(a[0]))
print(len(b[0]))
y_train_onehot=utils.to_categorical(labels_train[a[0]]-1)
y_test_onehot=utils.to_categorical(labels_test[b[0]]-1)
print(X1_train.shape)
print(iwc_train.shape)
#print(iwc_train.mean(axis=0))
import matplotlib.pyplot as plt

136573
67198
(561647, 112)
(561647, 50)


In [23]:
history = lidar_model.fit(
    X_train_scaled[a[0],10:62],y_train_onehot,
    validation_data=(X_test_scaled[b[0],10:62], y_test_onehot),
    batch_size=128,
    epochs=13,
    verbose=1)
lidar_model.save("lidar_Classif_Model.h5")


Epoch 1/13
1067/1067 [==============================] - 1s 922us/step - loss: 2.0063 - accuracy: 0.3633 - val_loss: 1.8752 - val_accuracy: 0.3854
Epoch 2/13
1067/1067 [==============================] - 1s 822us/step - loss: 1.8993 - accuracy: 0.3838 - val_loss: 1.8556 - val_accuracy: 0.3871
Epoch 3/13
1067/1067 [==============================] - 1s 867us/step - loss: 1.8785 - accuracy: 0.3866 - val_loss: 1.8492 - val_accuracy: 0.3875
Epoch 4/13
1067/1067 [==============================] - 1s 806us/step - loss: 1.8698 - accuracy: 0.3881 - val_loss: 1.8396 - val_accuracy: 0.3904
Epoch 5/13
1067/1067 [==============================] - 1s 778us/step - loss: 1.8611 - accuracy: 0.3894 - val_loss: 1.8366 - val_accuracy: 0.3903
Epoch 6/13
1067/1067 [==============================] - 1s 772us/step - loss: 1.8556 - accuracy: 0.3893 - val_loss: 1.8307 - val_accuracy: 0.3910
Epoch 7/13
1067/1067 [==============================] - 1s 774us/step - loss: 1.8513 - accuracy: 0.3912 - val_loss: 1.8286 -

In [24]:
history = radar_model.fit(
    X_train_scaled[a[0],62:],y_train_onehot,
    validation_data=(X_test_scaled[b[0],62:], y_test_onehot),
    batch_size=128,
    epochs=13,
    verbose=1)
lidar_model.save("radar_Classif_Model.h5")


Epoch 1/13
1067/1067 [==============================] - 1s 863us/step - loss: 1.7937 - accuracy: 0.3265 - val_loss: 1.5009 - val_accuracy: 0.3869
Epoch 2/13
1067/1067 [==============================] - 1s 784us/step - loss: 1.5565 - accuracy: 0.3692 - val_loss: 1.4684 - val_accuracy: 0.3959
Epoch 3/13
1067/1067 [==============================] - 1s 768us/step - loss: 1.5227 - accuracy: 0.3781 - val_loss: 1.4591 - val_accuracy: 0.3989
Epoch 4/13
1067/1067 [==============================] - 1s 786us/step - loss: 1.5093 - accuracy: 0.3812 - val_loss: 1.4500 - val_accuracy: 0.4017
Epoch 5/13
1067/1067 [==============================] - 1s 777us/step - loss: 1.4978 - accuracy: 0.3850 - val_loss: 1.4469 - val_accuracy: 0.4023
Epoch 6/13
1067/1067 [==============================] - 1s 778us/step - loss: 1.4902 - accuracy: 0.3871 - val_loss: 1.4457 - val_accuracy: 0.4032
Epoch 7/13
1067/1067 [==============================] - 1s 818us/step - loss: 1.4854 - accuracy: 0.3899 - val_loss: 1.4429 -

In [25]:
history = radiometer_model.fit(
    X_train_scaled[a[0],:10],y_train_onehot,
    validation_data=(X_test_scaled[b[0],:10], y_test_onehot),
    batch_size=128,
    epochs=13,
    verbose=1)
lidar_model.save("radiometer_Classif_Model.h5")

Epoch 1/13
1067/1067 [==============================] - 1s 845us/step - loss: 2.0278 - accuracy: 0.3005 - val_loss: 1.8620 - val_accuracy: 0.3415
Epoch 2/13
1067/1067 [==============================] - 1s 753us/step - loss: 1.9027 - accuracy: 0.3316 - val_loss: 1.8519 - val_accuracy: 0.3434
Epoch 3/13
1067/1067 [==============================] - 1s 766us/step - loss: 1.8857 - accuracy: 0.3357 - val_loss: 1.8476 - val_accuracy: 0.3448
Epoch 4/13
1067/1067 [==============================] - 1s 747us/step - loss: 1.8775 - accuracy: 0.3388 - val_loss: 1.8468 - val_accuracy: 0.3451
Epoch 5/13
1067/1067 [==============================] - 1s 754us/step - loss: 1.8731 - accuracy: 0.3391 - val_loss: 1.8433 - val_accuracy: 0.3453
Epoch 6/13
1067/1067 [==============================] - 1s 768us/step - loss: 1.8696 - accuracy: 0.3405 - val_loss: 1.8422 - val_accuracy: 0.3460
Epoch 7/13
1067/1067 [==============================] - 1s 768us/step - loss: 1.8666 - accuracy: 0.3422 - val_loss: 1.8404 -

In [17]:
# define function to evaluate accuracy excluding class 0
#y_pred=lidar_model.predict(X_test_scaled[:,10:62])

def accuracy_excl0(y_true, y_pred):
    n=y_pred.shape[0]
    nt=0
    nc=0
    for i in range(n):
        if y_true[i,0]>0.99:
           continue
        else:
            i1=np.argmax(y_pred[i,:])
            i2=np.argmax(y_true[i,:])
            nt+=1
            if i1==i2:
                nc+=1
    print(nt,nc)
    return nc/nt

In [26]:
history = lidarRadarRadiometer_model.fit(
    X_train_scaled[a[0],:],y_train_onehot,
    validation_data=(X_test_scaled[b[0],:], y_test_onehot),
    batch_size=128,
    epochs=13,
    verbose=1)
lidar_model.save("lidarRadarRadiometer_Classif_Model.h5")

Epoch 1/13
1067/1067 [==============================] - 1s 893us/step - loss: 1.4069 - accuracy: 0.5107 - val_loss: 1.0630 - val_accuracy: 0.6086
Epoch 2/13
1067/1067 [==============================] - 1s 820us/step - loss: 1.1414 - accuracy: 0.5877 - val_loss: 1.0251 - val_accuracy: 0.6240
Epoch 3/13
1067/1067 [==============================] - 1s 797us/step - loss: 1.1009 - accuracy: 0.6009 - val_loss: 1.0108 - val_accuracy: 0.6290
Epoch 4/13
1067/1067 [==============================] - 1s 815us/step - loss: 1.0800 - accuracy: 0.6086 - val_loss: 1.0014 - val_accuracy: 0.6322
Epoch 5/13
1067/1067 [==============================] - 1s 843us/step - loss: 1.0689 - accuracy: 0.6138 - val_loss: 0.9951 - val_accuracy: 0.6345
Epoch 6/13
1067/1067 [==============================] - 1s 836us/step - loss: 1.0572 - accuracy: 0.6152 - val_loss: 0.9886 - val_accuracy: 0.6366
Epoch 7/13
1067/1067 [==============================] - 1s 847us/step - loss: 1.0527 - accuracy: 0.6181 - val_loss: 0.9886 -

In [27]:
history = lidarRadar_model.fit(
    X_train_scaled[a[0],10:],y_train_onehot,
    validation_data=(X_test_scaled[b[0],10:], y_test_onehot),
    batch_size=128,
    epochs=13,
    verbose=1)
lidar_model.save("lidarRadar_Classif_Model.h5")

Epoch 1/13
1067/1067 [==============================] - 1s 886us/step - loss: 1.4643 - accuracy: 0.5008 - val_loss: 1.1201 - val_accuracy: 0.5916
Epoch 2/13
1067/1067 [==============================] - 1s 811us/step - loss: 1.1944 - accuracy: 0.5729 - val_loss: 1.0752 - val_accuracy: 0.6075
Epoch 3/13
1067/1067 [==============================] - 1s 789us/step - loss: 1.1527 - accuracy: 0.5855 - val_loss: 1.0669 - val_accuracy: 0.6084
Epoch 4/13
1067/1067 [==============================] - 1s 802us/step - loss: 1.1323 - accuracy: 0.5910 - val_loss: 1.0564 - val_accuracy: 0.6116
Epoch 5/13
1067/1067 [==============================] - 1s 791us/step - loss: 1.1199 - accuracy: 0.5937 - val_loss: 1.0514 - val_accuracy: 0.6134
Epoch 6/13
1067/1067 [==============================] - 1s 807us/step - loss: 1.1097 - accuracy: 0.5974 - val_loss: 1.0460 - val_accuracy: 0.6160
Epoch 7/13
1067/1067 [==============================] - 1s 802us/step - loss: 1.1050 - accuracy: 0.5980 - val_loss: 1.0453 -